### [让港区充满大家的欢笑声吧！](让港区充满大家的欢笑声吧！)

事先说明这是我在打舰娘游戏时朋友出的题。题中的Z18是只可爱舰娘（不过放图有点不合适就是了）。

感谢“M逍遥”在表述上的指正。题主已根据想要表达的原意做出修改。

题目如下：

Z18，让港区充满大家的欢笑声吧！

已知每个舰娘每天会随机遇到三个不同的舰娘，每次相遇仅能两两相遇，任意两天的相遇情况都独立，每一天中发生相遇的顺序随机。

Z18具有状态「欢笑」，该状态的拥有舰娘能够使遇到的舰娘也拥有该状态。在第一天将Z18加入本来有15个无状态的舰娘的港区，然后开始这一天的随机相遇。请问当港区中所有舰娘都具有状态「欢笑」的概率首次超过50%时是第几天。

我不清楚这道题是否表达严谨。本人也不仅是想要单纯一个答案。如果能够说明一下这道题涉及的知识以及学习的途径，我将不胜感激。（鞠躬）

----

✍️ 先捋一下题目的意思哈。一共有16只舰娘。第一天Z18将欢笑状态“传染”给另外3只舰娘，当天结束的时候共有4只欢笑的舰娘。第二天这4只舰娘可能只是在内部互相传染，也可能会传染给更多的舰娘，甚至是全部传染到。也就是说，每天结束时欢笑的舰娘数量是一个1到16上的概率分布，不妨记为$D_n$。$D_1$是：

欢笑舰娘数|1|2|3|4|5|6|$\cdots$|16
--|--|--|--|--|--|--|--|--
**概率**|0|0|0|**1**|0|0|$\cdots$|0

我们关心的是16只舰娘全被感染的概率即$D_n[16]$。这个概率随着$n$的增加应该是会增加且最后趋近于1的（群体免疫）。我们想知道当$D_n[16] > 0.5$时，$n$最小是几？

In [32]:
import duckdb
from collections import defaultdict
import numpy as np

In [33]:
sql = """
with recursive S1(s) as (  -- next-day scenarios resulting from a single happy ship (Z18)
    from range(1 << 16) as r(i)
    where i % 2 and bit_count(i) = 4), 
Sn(n, s, cnt) as ( -- next-day scenario counts resulting from n happy ships
    from (values(0, 0, 1::double))
    union all
    select n+1, Sn.s | ((S1.s << n) & ((1 << 16) - 1) | (S1.s >> (16 - n))), sum(cnt)
    from Sn cross join S1
    where n < 16
    group by all)
select n as i, bit_count(s) as j, sum(cnt) / 455 ** n as p
from Sn
group by i, j
order by i, j
"""

In [106]:
f1 = np.array([[1. if j == 4 else 0. for j in range(17)]])
t = defaultdict(float, (((i,j), p) for (i,j,p) in duckdb.sql(sql).fetchall()))
T = np.array([[t[i,j] for j in range(17)] for i in range(17)])

def prob_all(n: int):
    return (f1 @ np.linalg.matrix_power(T, n-1))[0, -1].item()

[prob_all(n) for n in range(1, 5)]

[0.0, 8.623555761118114e-06, 0.6251196309245242, 0.9824995734527541]

In [108]:
[prob_all(n) for n in range(1, 6)]

[0.0,
 8.623555761118114e-06,
 0.6251196309245242,
 0.9824995734527541,
 0.9993758691434783]

In [15]:
type(t[3,5])

float

In [78]:
Matrix([1,2,3,4])

Matrix([
[1],
[2],
[3],
[4]])

In [73]:
((f1 * P * D**(n-1) * P.inv())[-1]).subs(n, 1)

-16.0777511596680

In [68]:
D**0

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [56]:
eqn = (f1 * P * D**n * P.inv())[-1] - 0.5
nsolve(eqn, n, 2)

1.87725148508487

In [47]:
(f1 * P * D**n * P.inv())[-1] 

74558.1687465701*0.00150302052266718**n - 407.512469371554*0.0351843720888321**n + 0.999999999999987*1.00000000000002**n + 7.90880945754487e+15*1.12701999218967e-10**n + 294174343765505.0*1.22589473984026e-9**n + 4.24963078362239e+15*2.33321313883066e-11**n - 2.24743525950023e+15*3.17051778022588e-10**n - 28479908361.3184*4.29406232203865e-7**n + 955983687095.501*4.59916397713268e-8**n + 580903680.352262*5.17214355835299e-6**n - 1.01848180152393e+16*5.2510115476101e-11**n - 21289386840613.0*6.48021885040287e-9**n - 8109152.02010763*7.89490110375161e-5**n

In [69]:
D ** 0

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [42]:


# list of scenarios impacted by the 0th bit
scenarios_0th = [i for i in range(2 ** N_SHIPS) if i % 2 and i.bit_count() == 1 + CONTACTS]

def scenarios_ith(i: int) -> list[int]:
    """ list of senarios impacted by the ith bit """
    
    def rol(s: int, i: int=1) -> int:
        """ rotate s left by i bits """
        i %= N_SHIPS
        return (s << i) & (2 ** N_SHIPS - 1) | (s >> (N_SHIPS - i))
    
    return [rol(s, i) for s in scenarios_0th]


@cache
def scenarios_by(n: int):
    """ multiset (counter) of scenarios impacted by the lowest n bits """
    
    assert 0 <= n <= N_SHIPS
        
    if n:
        prod = product(scenarios_by(n-1).items(), scenarios_ith(n))
        return sum((Counter({s1 | s2: cnt}) for (s1, cnt), s2 in prod), Counter())
    else:
        return Counter([0])

Counter({30: 1,
         46: 1,
         54: 1,
         58: 1,
         78: 1,
         86: 1,
         90: 1,
         102: 1,
         106: 1,
         114: 1,
         142: 1,
         150: 1,
         154: 1,
         166: 1,
         170: 1,
         178: 1,
         198: 1,
         202: 1,
         210: 1,
         226: 1,
         15: 1,
         23: 1,
         27: 1,
         39: 1,
         43: 1,
         51: 1,
         71: 1,
         75: 1,
         83: 1,
         99: 1,
         135: 1,
         139: 1,
         147: 1,
         163: 1,
         195: 1})

In [34]:
scenarios_by.cache_clear()